In [ ]:
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""A binary to train CIFAR-10 using a single GPU.
Accuracy:
cifar10_train.py achieves ~86% accuracy after 100K steps (256 epochs of
data) as judged by cifar10_eval.py.
Speed: With batch_size 128.
System        | Step Time (sec/batch)  |     Accuracy
------------------------------------------------------------------
1 Tesla K20m  | 0.35-0.60              | ~86% at 60K steps  (5 hours)
1 Tesla K40m  | 0.25-0.35              | ~86% at 100K steps (4 hours)
Usage:
Please see the tutorial and website for how to download the CIFAR-10
data set, compile the program and train the model.
http://tensorflow.org/tutorials/deep_cnn/
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import time

import tensorflow as tf

import cifar10

FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('train_dir', '/tmp/cifar10_train',
                           """Directory where to write event logs """
                           """and checkpoint.""")
tf.app.flags.DEFINE_integer('max_steps', 1000000,
                            """Number of batches to run.""")
tf.app.flags.DEFINE_boolean('log_device_placement', False,
                            """Whether to log device placement.""")
tf.app.flags.DEFINE_integer('log_frequency', 10,
                            """How often to log results to the console.""")


def train():
  """Train CIFAR-10 for a number of steps."""
  with tf.Graph().as_default():
    global_step = tf.train.get_or_create_global_step()

    # Get images and labels for CIFAR-10.
    # Force input pipeline to CPU:0 to avoid operations sometimes ending up on
    # GPU and resulting in a slow down.
    with tf.device('/cpu:0'):
      images, labels = cifar10.distorted_inputs()

    # Build a Graph that computes the logits predictions from the
    # inference model.b
    logits = cifar10.inference(images)

    # Calculate loss.
    loss = cifar10.loss(logits, labels)

    # Build a Graph that trains the model with one batch of examples and
    # updates the model parameters.
    train_op = cifar10.train(loss, global_step)

    class _LoggerHook(tf.train.SessionRunHook):
      """Logs loss and runtime."""

      def begin(self):
        self._step = -1
        self._start_time = time.time()

      def before_run(self, run_context):
        self._step += 1
        return tf.train.SessionRunArgs(loss)  # Asks for loss value.

      def after_run(self, run_context, run_values):
        if self._step % FLAGS.log_frequency == 0:
          current_time = time.time()
          duration = current_time - self._start_time
          self._start_time = current_time

          loss_value = run_values.results
          print(FLAGS.batch_size)
          examples_per_sec = FLAGS.log_frequency * FLAGS.batch_size / duration
          sec_per_batch = float(duration / FLAGS.log_frequency)

          format_str = ('%s: step %d, loss = %.2f (%.1f examples/sec; %.3f '
                        'sec/batch)')
          print (format_str % (datetime.now(), self._step, loss_value,
                               examples_per_sec, sec_per_batch))

    with tf.train.MonitoredTrainingSession(
        checkpoint_dir=FLAGS.train_dir,
        hooks=[tf.train.StopAtStepHook(last_step=FLAGS.max_steps),
               tf.train.NanTensorHook(loss),
               _LoggerHook()],
        config=tf.ConfigProto(
            log_device_placement=FLAGS.log_device_placement)) as mon_sess:
      while not mon_sess.should_stop():
        mon_sess.run(train_op)


def main(argv=None):  # pylint: disable=unused-argument
  cifar10.maybe_download_and_extract()
  if tf.gfile.Exists(FLAGS.train_dir):
    tf.gfile.DeleteRecursively(FLAGS.train_dir)
  tf.gfile.MakeDirs(FLAGS.train_dir)
  train()


if __name__ == '__main__':
  tf.app.run()

(128, 32, 3)


Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: input_producer/input_producer_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](input_producer, input_producer/RandomShuffle)]]

(128, 4, 32, 32, 3)
conv1
	(128, 4, 32, 32, 3) --> (128, 4, 32, 32, 64)
pool1
	(128, 4, 32, 32, 64) --> (128, 4, 16, 16, 64)


c:\users\cindy\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


norm1
	(128, 4, 16, 16, 64) --> (128, 4, 16, 16, 64)
conv2
	(128, 4, 16, 16, 64) --> (128, 4, 16, 16, 64)
norm2
	(128, 4, 16, 16, 64) --> (128, 4, 16, 16, 64)
pool2
	(128, 4, 16, 16, 64) --> (128, 4, 8, 8, 64)
conv2
	(128, 4, 8, 8, 64) --> (128, 4, 8, 8, 64)
norm3
	(128, 4, 8, 8, 64) --> (128, 4, 8, 8, 64)
pool3
	(128, 4, 8, 8, 64) --> (128, 4, 4, 4, 64)
conv4
	(128, 4, 4, 4, 64) --> (128, 4, 4, 4, 64)
norm3
	(128, 4, 4, 4, 64) --> (128, 4, 4, 4, 64)
pool4
	(128, 4, 4, 4, 64) --> (128, 4, 2, 2, 64)
conv5
	(128, 4, 2, 2, 64) --> (128, 4, 2, 2, 64)
norm5
	(128, 4, 2, 2, 64) --> (128, 4, 4, 4, 64)
pool5
	(128, 4, 4, 4, 64) --> (128, 4, 2, 2, 64)
INFO:tensorflow:Summary name conv1/weight_loss (raw) is illegal; using conv1/weight_loss__raw_ instead.
INFO:tensorflow:Summary name conv2/weight_loss (raw) is illegal; using conv2/weight_loss__raw_ instead.
INFO:tensorflow:Summary name conv3/weight_loss (raw) is illegal; using conv3/weight_loss__raw_ instead.
INFO:tensorflow:Summary name conv4/we

128
2018-06-02 22:30:03.159963: step 670, loss = 1.75 (430.0 examples/sec; 0.298 sec/batch)
128
2018-06-02 22:30:06.138998: step 680, loss = 1.49 (429.7 examples/sec; 0.298 sec/batch)
128
2018-06-02 22:30:09.120025: step 690, loss = 1.82 (429.2 examples/sec; 0.298 sec/batch)
INFO:tensorflow:global_step/sec: 3.20175
128
2018-06-02 22:30:13.444924: step 700, loss = 1.72 (296.7 examples/sec; 0.431 sec/batch)
128
2018-06-02 22:30:16.439915: step 710, loss = 1.76 (425.8 examples/sec; 0.301 sec/batch)
128
2018-06-02 22:30:19.427968: step 720, loss = 1.61 (428.4 examples/sec; 0.299 sec/batch)
128
2018-06-02 22:30:22.443904: step 730, loss = 1.77 (424.4 examples/sec; 0.302 sec/batch)
128
2018-06-02 22:30:25.441856: step 740, loss = 1.56 (427.0 examples/sec; 0.300 sec/batch)
128
2018-06-02 22:30:28.441834: step 750, loss = 1.58 (426.7 examples/sec; 0.300 sec/batch)
128
2018-06-02 22:30:31.463579: step 760, loss = 1.43 (423.6 examples/sec; 0.302 sec/batch)
128
2018-06-02 22:30:34.463497: step 77

128
2018-06-02 22:34:31.229408: step 1520, loss = 1.05 (424.4 examples/sec; 0.302 sec/batch)
128
2018-06-02 22:34:34.254321: step 1530, loss = 1.08 (423.0 examples/sec; 0.303 sec/batch)
128
2018-06-02 22:34:37.241553: step 1540, loss = 0.98 (428.5 examples/sec; 0.299 sec/batch)
128
2018-06-02 22:34:40.222301: step 1550, loss = 1.19 (429.6 examples/sec; 0.298 sec/batch)
128
2018-06-02 22:34:43.204446: step 1560, loss = 1.23 (429.1 examples/sec; 0.298 sec/batch)
128
2018-06-02 22:34:46.197484: step 1570, loss = 1.10 (427.8 examples/sec; 0.299 sec/batch)
128
2018-06-02 22:34:49.187495: step 1580, loss = 1.03 (428.0 examples/sec; 0.299 sec/batch)
128
2018-06-02 22:34:52.189763: step 1590, loss = 1.05 (426.5 examples/sec; 0.300 sec/batch)
INFO:tensorflow:global_step/sec: 3.20767
128
2018-06-02 22:34:56.345925: step 1600, loss = 1.06 (308.3 examples/sec; 0.415 sec/batch)
128
2018-06-02 22:34:59.338869: step 1610, loss = 1.00 (427.0 examples/sec; 0.300 sec/batch)
128
2018-06-02 22:35:02.33933

128
2018-06-02 22:38:54.002234: step 2360, loss = 1.21 (427.4 examples/sec; 0.299 sec/batch)
128
2018-06-02 22:38:56.992108: step 2370, loss = 0.83 (428.1 examples/sec; 0.299 sec/batch)
128
2018-06-02 22:38:59.977477: step 2380, loss = 0.92 (428.9 examples/sec; 0.298 sec/batch)
128
2018-06-02 22:39:02.953628: step 2390, loss = 0.77 (430.1 examples/sec; 0.298 sec/batch)
INFO:tensorflow:global_step/sec: 3.21212
128
2018-06-02 22:39:07.204426: step 2400, loss = 0.78 (302.0 examples/sec; 0.424 sec/batch)
128
2018-06-02 22:39:10.185588: step 2410, loss = 0.78 (427.4 examples/sec; 0.300 sec/batch)
128
2018-06-02 22:39:13.158135: step 2420, loss = 0.82 (430.6 examples/sec; 0.297 sec/batch)
128
2018-06-02 22:39:16.146629: step 2430, loss = 0.80 (428.3 examples/sec; 0.299 sec/batch)
128
2018-06-02 22:39:19.121903: step 2440, loss = 0.67 (430.2 examples/sec; 0.298 sec/batch)
128
2018-06-02 22:39:22.140534: step 2450, loss = 0.68 (424.2 examples/sec; 0.302 sec/batch)
128
2018-06-02 22:39:25.15727